In [2]:
import cv2
import os
from multiprocessing import Pool
from decord import VideoReader
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:



# handle a single file
# source of video file:
path_to_file = r"C:\Users\gutzc\GitHub\human_micro_gesture_classifier\miga_dataset\testing split files\Sample0001_color.mp4"

# source of labels file:
path_to_labels = r"C:\Users\gutzc\GitHub\human_micro_gesture_classifier\miga_dataset\testing split files\Sample0001_finelabels.csv"



# define parameters
WIN_LEN = 64  # 2 seconds
N_CLASSES = 17


In [6]:
def get_indecies(path_to_file, path_to_labels, vr=None):
    # load the file
    if vr is None:
        vr = VideoReader(path_to_file)
    n_frames = len(vr)
    
    # load csv file and get variables
    df = pd.read_csv(path_to_labels, header=None)
    df.columns = ['labels','start_frame','end_frame']
    labels_list = df['labels'].values
    start_frame_list = df['start_frame'].values
    end_frame_list = df['end_frame'].values
    duration_list = end_frame_list - start_frame_list
    
    n_labels = len(labels_list)
    
    
    # subtracts 1 from labels to shift 1-17 -> 0-16
    labels_list = labels_list - 1
    
    # combine frame getter
    list_of_label_lists = []
    list_of_onehots = []
    list_of_frame_lists = []
    new_start_inds = []
    new_end_inds = []
    list_of_file_names = []
    
    i = 0
    duration_offset = duration_list[i]//2 
    if duration_offset > WIN_LEN//2:
        duration_offset = WIN_LEN//2 # start from the first frame
    
    # get first ind
    start_frame = max([start_frame_list[i] + duration_offset - WIN_LEN//2, 0])
    end_frame = min([start_frame+WIN_LEN, n_frames])
    j=0
    i = 0
    while i < n_labels -1:
        
        
        # init labels
        new_labels_list = [labels_list[i]]
        
        # init onehot
        onehot = np.zeros(N_CLASSES)
        onehot[labels_list[i]] = 1
        
            # check if there is a new label to append
        k = i + 1
        while k < n_labels and start_frame_list[k] < end_frame:
            # add that labels to the list of the current ONLY if this is new
            if labels_list[k] not in new_labels_list:
                new_labels_list.append(labels_list[k]) 
                onehot[labels_list[k]] = 1
            k += 1
        list_of_label_lists.append(new_labels_list)
        list_of_onehots.append(onehot)
        this_frames_list = list(range(start_frame, end_frame))
        if len(this_frames_list) < WIN_LEN:
                # just ignore the last 2 seconds
            # this_frames_list += [this_frames_list[-1]]*(WIN_LEN - len(this_frames_list))
            break
            
        this_file_name_base = os.path.basename(path_to_file).split('.')[0]
        this_file_name = f'{this_file_name_base}_{j:03}_{start_frame:05}-{end_frame:05}.mp4'
        list_of_file_names.append(this_file_name)
        new_start_inds.append(start_frame)
        new_end_inds.append(end_frame)
        list_of_frame_lists.append(this_frames_list)
        
        
        # if the video was longer than 64 frames, splitting until finished
        if end_frame < end_frame_list[i]:
            start_frame = end_frame
            end_frame = min([end_frame+WIN_LEN,n_frames])
        else:
            i += 1
            duration_offset = duration_list[i]//2 
            if duration_offset > WIN_LEN//2:
                duration_offset = WIN_LEN//2 # start from the first frame
            
            # get first ind
            start_frame = max([start_frame_list[i] + duration_offset - WIN_LEN//2, 0])
            end_frame = min([start_frame+WIN_LEN, n_frames])
            
        j += 1
    # combine everything into a dataframe
    df_out = pd.DataFrame(zip(list_of_file_names,
                              new_start_inds,
                              new_end_inds,
                              list_of_label_lists,
                              list_of_onehots,
                            list_of_frame_lists                      
                        ), columns=['file_name', 'start_frame', 'end_frame', 'labels', 'multihot_labels','frame_numbers'])
    # add folder name and full path name
    df_out['source_folder'] = os.path.dirname(path_to_file)
    df_out['source_video'] = os.path.basename(path_to_file)
    return df_out

def save_video(output_video_path, frames,fps):


    # Create a directory to save frames
    if not os.path.exists(output_video_path):
        os.makedirs(output_video_path)
    frames_list = [frame for frame in frames]
    # Determine the frame size from the first frame
    height, width , _= frames_list[0].shape 

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Change codec as per your requirement
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Write frames to the video
    for frame in frames_list:
        out.write(frame)

    # Release the VideoWriter object
    out.release()


# save the files and labels
def save_files(df_in: pd.DataFrame, vr:VideoReader, destination_folder):
    for _, row in tqdm(df_in.iterrows(), total=len(df_in)):
        file_name = row['file_name']
        frame_numbers = row['frame_numbers']
        frames = vr.get_batch(frame_numbers)
        output_video_path = os.path.join(destination_folder, file_name) 
        save_video(output_video_path, frames,fps=30)
        
# save_files(df_in=df_out, vr=vr, destination_folder=destination_folder)
# df_out = get_indecies(path_to_file, path_to_labels, vr=None)
destination_folder = os.path.dirname(path_to_file)
vr = VideoReader(path_to_file)
df_out = get_indecies(path_to_file, path_to_labels, vr=vr)
# save_files(df_in=df_out, vr=vr, destination_folder=destination_folder)

In [14]:


destination_folder = os.path.dirname(path_to_file)
vr = VideoReader(path_to_file)
df_out = get_indecies(path_to_file, path_to_labels, vr=vr)
save_files(df_in=df_out, vr=vr, destination_folder=destination_folder)

  0%|          | 0/48 [00:00<?, ?it/s]


OSError: [WinError -529697949] Windows Error 0xe06d7363

In [ ]:
import cv2
import pandas as pd
import numpy as np
import os

def get_indices(path_to_file, path_to_labels):
    # Load csv file and get variables
    df = pd.read_csv(path_to_labels, header=None)
    df.columns = ['labels', 'start_frame', 'end_frame']
    labels_list = df['labels'].values
    start_frame_list = df['start_frame'].values
    end_frame_list = df['end_frame'].values
    duration_list = end_frame_list - start_frame_list

    # Subtract 1 from labels to shift 1-17 -> 0-16
    labels_list = labels_list - 1

    # Define parameters
    WIN_LEN = 64  # 2 seconds
    N_CLASSES = 17

    # Combine frame getter
    list_of_label_lists = []
    list_of_onehots = []
    list_of_frame_lists = []
    new_start_inds = []
    new_end_inds = []
    list_of_file_names = []

    n_labels = len(labels_list)
    n_frames = int(cv2.VideoCapture(path_to_file).get(cv2.CAP_PROP_FRAME_COUNT))

    i = 0
    duration_offset = duration_list[i] // 2
    if duration_offset > WIN_LEN // 2:
        duration_offset = WIN_LEN // 2  # start from the first frame

    j = 0
    while i < n_labels - 1:
        # Get first index
        start_frame = max([start_frame_list[i] + duration_list[i] // 2 - WIN_LEN // 2, 0])
        end_frame = min([start_frame + WIN_LEN, n_frames])

        # Initialize lists
        frames_list = [start_frame]
        new_labels_list = [labels_list[i]]
        onehot = np.zeros(N_CLASSES)
        onehot[labels_list[i]] = 1

        # Check for new labels to append
        k = i + 1
        while k < n_labels and start_frame_list[k] < end_frame:
            if labels_list[k] not in new_labels_list:
                new_labels_list.append(labels_list[k])
                onehot[labels_list[k]] = 1
            k += 1

        list_of_label_lists.append(new_labels_list)
        list_of_onehots.append(onehot)
        this_frames_list = list(range(start_frame, end_frame))
        
        if len(this_frames_list) < WIN_LEN:
            break

        this_file_name_base = os.path.basename(path_to_file).split('.')[0]
        this_file_name = f'{this_file_name_base}_{j:03}_{start_frame:05}-{end_frame:05}.mp4'
        list_of_file_names.append(this_file_name)
        new_start_inds.append(start_frame)
        new_end_inds.append(end_frame)
        list_of_frame_lists.append(this_frames_list)

        # If the video was longer than 64 frames, continue splitting
        if end_frame < end_frame_list[i]:
            start_frame = end_frame
            end_frame = min([end_frame + WIN_LEN, n_frames])
        else:
            i += 1
            duration_offset = duration_list[i] // 2
            if duration_offset > WIN_LEN // 2:
                duration_offset = WIN_LEN // 2  # start from the first frame

            # Get first index for the next label
            start_frame = max([start_frame_list[i] + duration_list[i] // 2 - WIN_LEN // 2, 0])
            end_frame = min([start_frame + WIN_LEN, n_frames])

        j += 1

    # Combine everything into a DataFrame
    df_out = pd.DataFrame(zip(list_of_file_names,
                              new_start_inds,
                              new_end_inds,
                              list_of_label_lists,
                              list_of_onehots,
                              list_of_frame_lists
                              ),
                          columns=['file_name', 'start_frame', 'end_frame', 'labels', 'multihot_labels',
                                   'frame_numbers'])
    # Add folder name and full path name
    df_out['source_folder'] = os.path.dirname(path_to_file)
    df_out['source_video'] = os.path.basename(path_to_file)
    return df_out

def save_video(output_video_path, frames, fps=30):
    # Determine the frame size from the first frame
    height, width, _ = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Change codec as per your requirement
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Write frames to the video
    for frame in frames:
        out.write(frame)

    # Release the VideoWriter object
    out.release()

# Save the files and labels
def save_files(df_in, path_to_file):
    vr = cv2.VideoCapture(path_to_file)
    for _, row in df_in.iterrows():
        file_name = row['file_name']
        frame_numbers = row['frame_numbers']
        frames = []
        for frame_number in frame_numbers:
            vr.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = vr.read()
            if ret:
                frames.append(frame)
            else:
                print(f"Error reading frame {frame_number} from video {path_to_file}")
        output_video_path = os.path.join(row['source_folder'], file_name)
        save_video(output_video_path, frames)
    vr.release()

# Example usage:
path_to_file = r"C:\Users\gutzc\GitHub\human_micro_gesture_classifier\miga_dataset\testing split files\Sample0001_color.mp4"
path_to_labels = r"C:\Users\gutzc\GitHub\human_micro_gesture_classifier\miga_dataset\testing split files\Sample0001_finelabels.csv"
df_out = get_indices(path_to_file, path_to_labels)
save_files(df_out, path_to_file)


In [8]:

def get_indices(path_to_file, path_to_labels, n_frames):
    # load csv file and get variables
    df = pd.read_csv(path_to_labels, header=None)
    df.columns = ['labels', 'start_frame', 'end_frame']
    labels_list = df['labels'].values
    start_frame_list = df['start_frame'].values
    end_frame_list = df['end_frame'].values
    duration_list = end_frame_list - start_frame_list

    n_labels = len(labels_list)

    # subtracts 1 from labels to shift 1-17 -> 0-16
    labels_list = labels_list - 1

    # combine frame getter
    list_of_label_lists = []
    list_of_onehots = []
    list_of_frame_lists = []
    new_start_inds = []
    new_end_inds = []
    list_of_file_names = []

    i = 0
    duration_offset = duration_list[i] // 2
    if duration_offset > WIN_LEN // 2:
        duration_offset = WIN_LEN // 2  # start from the first frame

    # get first ind
    start_frame = max([start_frame_list[i] + duration_offset - WIN_LEN // 2, 0])
    end_frame = min([start_frame + WIN_LEN, n_frames])
    j = 0
    i = 0
    while i < n_labels - 1:

        # init labels
        new_labels_list = [labels_list[i]]

        # init onehot
        onehot = np.zeros(N_CLASSES)
        onehot[labels_list[i]] = 1

        # check if there is a new label to append
        k = i + 1
        while k < n_labels and start_frame_list[k] < end_frame:
            # add that labels to the list of the current ONLY if this is new
            if labels_list[k] not in new_labels_list:
                new_labels_list.append(labels_list[k])
                onehot[labels_list[k]] = 1
            k += 1
        list_of_label_lists.append(new_labels_list)
        list_of_onehots.append(onehot)
        this_frames_list = list(range(start_frame, end_frame))
        if len(this_frames_list) < WIN_LEN:
            # just ignore the last 2 seconds
            # this_frames_list += [this_frames_list[-1]]*(WIN_LEN - len(this_frames_list))
            break

        this_file_name_base = os.path.basename(path_to_file).split('.')[0]

        new_start_inds.append(start_frame)
        new_end_inds.append(end_frame)
        list_of_frame_lists.append(this_frames_list)

        # if the video was longer than 64 frames, splitting until finished
        if end_frame < end_frame_list[i]:
            start_frame = end_frame
            end_frame = min([end_frame + WIN_LEN, n_frames])
        else:
            i += 1
            duration_offset = duration_list[i] // 2
            if duration_offset > WIN_LEN // 2:
                duration_offset = WIN_LEN // 2  # start from the first frame

            # get first ind
            start_frame = max([start_frame_list[i] + duration_offset - WIN_LEN // 2, 0])
            end_frame = min([start_frame + WIN_LEN, n_frames])

        labels_string = '_'.join([str(int(i)) for i in new_labels_list])
        this_file_name = f'{this_file_name_base}_{j:03}_{start_frame:05}-{end_frame:05}_{labels_string}.mp4'
        list_of_file_names.append(this_file_name)
        j += 1
    # combine everything into a dataframe
    df_out = pd.DataFrame(zip(list_of_file_names,
                              new_start_inds,
                              new_end_inds,
                              list_of_label_lists,
                              list_of_onehots,
                              list_of_frame_lists
                              ), columns=['file_name', 'start_frame', 'end_frame', 'labels', 'multihot_labels',
                                          'frame_numbers'])
    # add folder name and full path name
    df_out['source_folder'] = os.path.dirname(path_to_file)
    df_out['source_video'] = os.path.basename(path_to_file)
    return df_out
df_out_1 = get_indices(path_to_file, path_to_labels, n_frames=22000)


In [10]:
df_out_1.loc[df_out_1['labels'].apply(lambda x: len(x))>1]

,file_name,start_frame,end_frame,labels,multihot_labels,frame_numbers,source_folder,source_video
50,Sample0001_color_050_21516-21580_11_7.mp4,21489,21553,"[11, 7]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[21489, 21490, 21491, 21492, 21493, 21494, 214...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4


In [7]:
df_out

,file_name,start_frame,end_frame,labels,multihot_labels,frame_numbers,source_folder,source_video
0,Sample0001_color_000_00100-00164.mp4,100,164,[14],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[100, 101, 102, 103, 104, 105, 106, 107, 108, ...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
1,Sample0001_color_001_00164-00228.mp4,164,228,[14],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[164, 165, 166, 167, 168, 169, 170, 171, 172, ...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
2,Sample0001_color_002_00831-00895.mp4,831,895,[11],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[831, 832, 833, 834, 835, 836, 837, 838, 839, ...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
3,Sample0001_color_003_01697-01761.mp4,1697,1761,[11],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1697, 1698, 1699, 1700, 1701, 1702, 1703, 170...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
4,Sample0001_color_004_04835-04899.mp4,4835,4899,[7],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[4835, 4836, 4837, 4838, 4839, 4840, 4841, 484...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
5,Sample0001_color_005_06317-06381.mp4,6317,6381,[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[6317, 6318, 6319, 6320, 6321, 6322, 6323, 632...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
6,Sample0001_color_006_06437-06501.mp4,6437,6501,[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[6437, 6438, 6439, 6440, 6441, 6442, 6443, 644...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
7,Sample0001_color_007_06501-06565.mp4,6501,6565,[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[6501, 6502, 6503, 6504, 6505, 6506, 6507, 650...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
8,Sample0001_color_008_06547-06611.mp4,6547,6611,[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[6547, 6548, 6549, 6550, 6551, 6552, 6553, 655...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
9,Sample0001_color_009_06611-06675.mp4,6611,6675,[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[6611, 6612, 6613, 6614, 6615, 6616, 6617, 661...",C:\Users\gutzc\GitHub\human_micro_gesture_clas...,Sample0001_color.mp4
